In [1]:
#! /usr/bin/env python3
import os
import sys
import re
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fftpack import fft

import matplotlib as mpl
from matplotlib import cm

from collections import OrderedDict
from matplotlib.colors import ListedColormap, LinearSegmentedColormap


cmaps = OrderedDict()

In [2]:
FILE = 'GEN3CH_4_009.dig'

In [3]:
# coding:utf-8

"""
::

   Author:  LANL Clinic 2019 --<lanl19@cs.hmc.edu>
   Purpose: Compute a spectrogram from a DigFile
   Created: 9/20/19
"""

import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

from digfile import DigFile


class Spectrogram:
    """

    A Spectrogram takes a DigFile and a time range, as
    well as plenty of options, and generates a spectrogram
    using scipy.signal.spectrogram.

    Required arguments to the constructor:
        digfile: either an instance of DigFile or the filename of a .dig file

    **Optional arguments and their default values**

    t_start: (digfile.t0) time of the first point to use in the spectrogram
    ending:  (None) either the time of the last point or a positive integer
             representing the number of points to use; if None, the final
             point in the digfile is used.
    wavelength: (1550.0e-9) the wavelength in meters
    points_per_spectrum: (8192) the number of values used to generate
        each spectrum. Should be a power of 2.
    overlap: (1/4) the fraction of points_per_spectrum to overlap in
        successive spectra. An overlap of 0 means that each sample is used
        in only one spectrum. The default value means that successive
        spectra share 1/4 of their source samples.
    window_function: (None) the window function used by signal.spectrogram.
        The default value implies a ('tukey', 0.25) window.
    form: ('db') whether to use power values ('power'), decibels ('db'),
        or log(power) ('log') in reporting spectral intensities.
    convert_to_voltage: (True) scale the integer values stored in the
        .dig file to voltage before computing the spectrogram. If False,
        the raw integral values are used.
    detrend: ("linear") the background subtraction method.

    **Computed fields**

    t:         array of times at which the spectra are computed
    f:         array of frequencies present in each spectrum
    v:         array of velocities corresponding to each spectrum
    intensity: two-dimensional array of (scaled) intensities, which
               is the spectrogram. The first index corresponds to
               frequency/velocity, the second to time.
    """

    _fields = ("points_per_spectrum",
               "overlap",
               "window_function",
               "form",
               "use_voltage",
               "detrend")

    def __init__(self,
                 digfile,
                 t_start=None,
                 ending=None,
                 wavelength=1550.0e-9,
                 points_per_spectrum=8192,
                 overlap=0.25,
                 window_function=None,  # 'hanning',
                 form='db',
                 convert_to_voltage=True,
                 detrend="linear",
                 **kwargs
                 ):
        """
        TODO: We are currently not handling kwargs
        """
        if isinstance(digfile, str):
            digfile = DigFile(digfile)
        if isinstance(digfile, DigFile):
            self.data = digfile
        else:
            raise TypeError("Unknown file type")

        self.t_start = t_start if t_start != None else self.data.t0
        p_start, p_end = self.data._points(self.t_start, ending)
        self.t_end = self.t_start + self.data.dt * (p_end - p_start + 1)

        self.wavelength = wavelength
        self.points_per_spectrum = points_per_spectrum
        self.overlap = overlap
        self.window_function = window_function
        self.form = form
        self.use_voltage = convert_to_voltage
        self.detrend = detrend

        # the following will be set by _calculate
        self.time = None
        self.frequency = None
        self.velocity = None
        self.intensity = None

        # deal with kwargs

        try:
            if False:
                self._load()
            else:
                raise Exception()
        except:
            self._compute(ending)
            # self._save()

    def _compute(self, ending):
        """
        Compute a spectrogram. This needs work! There need to be
        lots more options that we either want to supply with
        default values or decode kwargs. But it should be a start.
        """
        if self.use_voltage:
            vals = self.data.values(self.t_start, ending)
        else:
            vals = self.data.raw_values(self.t_start, ending)

        # if normalize:
        #    vals = self.normalize(
        #        vals, chunksize=fftSize, remove_dc=remove_dc)
        freqs, times, spec = signal.spectrogram(
            vals,
            1.0 / self.data.dt,  # the sample frequency
            window=self.window_function if self.window_function else (
                'tukey', 0.25),
            nperseg=self.points_per_spectrum,
            noverlap=int(self.overlap * self.points_per_spectrum),
            detrend=self.detrend,  # could be constant,
            scaling="spectrum"
        )
        times += self.t_start
        # Attempt to deduce baselines
        # baselines = np.sum(spec, axis=1)

        # Convert to a logarithmic representation and use floor to attempt
        # to suppress some noise.
        spec *= 2.0 / (self.points_per_spectrum * self.data.dt)
        epsilon = 1e-10  # use this to suppress the divide by zero warnings
        if self.form == 'db':
            spec = 20 * np.log10(spec + epsilon)
        elif self.form == 'log':
            spec = np.log10(spec + epsilon)
        self.intensity = spec  # a two-dimensional array
        # the first index is frequency, the second time

        self.frequency = freqs
        self.time = times

        # scale the frequency axis to velocity
        self.velocity = freqs * 0.5 * self.wavelength  # velocities

    def set(self, **kwargs):
        """
        Update the spectrogram to use the new parameters
        specified in the keyword arguments. If any changes cause
        the underlying values to change, recompute the spectrogram.
        """
        changed = False
        for field in self._fields:
            if field in kwargs and kwargs[field] != getattr(self, field):
                changed = True
                setattr(self, field, kwargs[field])
        if changed:
            self._compute(None)

    def __str__(self):
        return ", ".join(
            [str(x) for x in
             [self.data.filename,
              f"{self.points_per_spectrum} / {self.shift}",
              self.form,
              self.intensity.shape
              ]
             ])

    def _point_to_time(self, p):
        "Map a point index to a time"
        return self.time[p]

    def _time_to_index(self, t):
        "Map a time to a point number"
        p = (t - self.t_start) / (self.time[1] - self.time[0])
        p = int(0.5 + p)  # round to an integer
        if p < 0:
            return 0
        return min(p, len(self.time) - 1)

    def _velocity_to_index(self, v):
        "Map a velocity value to a point number"
        p = (v - self.velocity[0]) / (self.velocity[1] - self.velocity[0])
        p = int(0.5 + p)  # round
        if p < 0:
            return 0
        return min(p, -1 + len(self.velocity))

    def slice(self, time_range, velocity_range):
        """
        Input:
            time_range: Array/Tuple/List of times (t0, t1)
                t0 should be greater than t1 but we will handle the other case
            velocity_range: Array/Tuple/List of velocities (v0, v1)
                v0 should be greater than v1 but we will handle the other case
        Output:
            3 arrays time, velocity, intensity
            time: the time values used in the measurement from t0 to t1 inclusive.
            velocity: the velocity values measured from v0 to v1 inclusive.
            intensity: the corresponding intensity values that we measured.
        """
        if time_range == None:
            time0, time1 = 0, len(self.time) - 1
        else:
            time0, time1 = [self._time_to_index(t) for t in time_range]
        if velocity_range == None:
            vel0, vel1 = 0, len(self.velocity) - 1
        else:
            vel0, vel1 = [self._velocity_to_index(v) for v in velocity_range]
        if time0 > time1:
            # Then we will just swap them.
            tmp = time0
            time0 = time1
            time1 = time0
        if vel0 > vel1:
            # Then we will just swap them so that we can index normally.
            tmp = vel0
            vel0 = vel1
            vel1 = tmp
        tvals = self.time[time0:time1 + 1]
        vvals = self.velocity[vel0:vel1 + 1]
        ivals = self.intensity[vel0:vel1+1, time0:time1+1]
        return tvals, vvals, ivals


    # Routines to archive the computed spectrogram and reload from disk

    def _location(self, location, create=False):
        """

        """
        if location == "":
            location = os.path.splitext(self.data.path)[0] + \
                '.spectrogram'
        if os.path.exists(location) and not os.path.isdir(location):
            raise FileExistsError
        if not os.path.exists(location) and create:
            os.mkdir(location)
        return location

    def _save(self, location=""):
        """
        Save a representation of this spectrogram.
        The format is a folder holding the three numpy arrays
        and a text file with the parameters.
        If the location is a blank string, the folder has
        the name of the digfile, with .dig replaced by .spectrogram.
        """
        location = self._location(location, True)
        with open(os.path.join(location, "properties"), 'w') as f:
            for field in self._fields:
                f.write(f"{field}\t{getattr(self,field)}\n")
        np.savez_compressed(
            os.path.join(location, "vals"),
            velocity=self.velocity,
            frequency=self.frequency,
            time=self.time,
            intensity=self.intensity)

    def _load(self, location=""):
        location = self._location(location)
        if not os.path.isdir(location):
            raise FileNotFoundError
        try:
            with open(os.path.join(
                    location, "properties"), 'r') as f:
                for line in f.readlines():
                    field, value = line.split('\t')
                    assert value == getattr(field)
            loaded = np.load(os.path.join(location, "vals"))
            for k, v in loaded.items():
                setattr(self, k, v)
            return True
        except Exception as eeps:
            print(eeps)
        return False

    @property
    def max(self):
        """The maximum intensity value"""
        return self.intensity.max()

    @property
    def v_max(self):
        return self.wavelength * 0.25 / self.data.dt

    def power(self, values):
        """
        Given an np.array of intensity values from the spectrogram,
        return the corresponding power values (undoing any logarithms,
        if necessary).
        """
        if self.form == 'db':
            return np.power(10.0, 0.05 * values)
        if self.form == 'log':
            return np.power(10.0, values)
        return values

    def plot(self, axes=None, **kwargs):
        # max_vel=6000, vmin=-200, vmax=100):
        if axes == None:
            axes = plt.gca()
        if 'max_vel' in kwargs:
            axes.set_ylim(top=kwargs['max_vel'])
            del kwargs['max_vel']
        if 'max_time' in kwargs:
            axes.set_xlim(right=kwargs['max_time'])
            del kwargs['max_time']

        pcm = axes.pcolormesh(
            self.time * 1e6,
            self.velocity,
            self.intensity,
            **kwargs)

        plt.gcf().colorbar(pcm, ax=axes)
        axes.set_ylabel('Velocity (m/s)')
        axes.set_xlabel('Time ($\mu$s)')
        title = self.data.filename.split('/')[-1]
        axes.set_title(title.replace("_", "\\_"))

#         if 'additonalPlot'

#             fig = plt.figure()
#             plt.plot(sgram['t'], bestVelocity, color="red")
        
        return pcm
    
    def getDT(self):
        return self.dt


if False:
    def scan_data(self):
        """
        Load the entire file and determine the range of raw integer values
        """
        raw = self.raw_values(self.t0, self.num_samples)
        self.raw = dict(min=np.min(raw), max=np.max(raw), mean=np.mean(raw))
        self.raw['range'] = self.raw['max'] - self.raw['min']
#         instrument_spec_codes = {
#             'BYT_N': 'binary_data_field_width',
#             'BIT_N': 'bits',
#             'ENC': 'encoding',
#             'BN_F': 'number_format',
#             'BYT_O': 'byte_order',
#             'WFI': 'source_trace',
#             'NR_P': 'number_pixel_bins',
#             'PT_F': 'point_format',
#             'XUN': 'x_unit',
#             'XIN': 'x_interval',
#             'XZE': 'post_trigger_seconds',
#             'PT_O': 'pulse_train_output',
#             'YUN': 'y_unit',
#             'YMU': 'y_scale_factor',
#             'YOF': 'y_offset',
#             'YZE': 'y_component',
#             'NR_FR': 'NR_FR'

    def __str__(self):
        return "\n".join([
            self.filename,
            f"{self.bits} bits" +
            f" {self.notes['byte_order']} first" if 'byte_order' in self.notes else "",
            f"{self.t0*1e6} µs to {(self.t0 + self.dt*self.num_samples)*1e6} µs in steps of {self.dt*1e12} ps"
        ])

    def normalize(self, array, chunksize=4096, remove_dc=True):
        """
        Given an array of periodically sampled points, normalize to a
        peak amplitude of 1, possibly after removing dc in segments of
        chunksize.
        """
        if remove_dc:
            num_chunks, num_leftovers = divmod(len(array), chunksize)
            if num_leftovers:
                leftovers = array[-num_leftovers:]
                chunks = array[:num_chunks *
                               chunksize].reshape((num_chunks, chunksize))
                avg = np.mean(leftovers)
                leftovers -= avg
            else:
                chunks = array.reshape((num_chunks, chunksize))
            # compute the average of each chunk
            averages = np.mean(chunks, axis=1)
            # shift each chunk to have zero mean
            for n in range(num_chunks):
                chunks[n, :] -= averages[n]
            flattened = chunks.reshape(num_chunks * chunksize)
            if num_leftovers:
                flattened = np.concatenate((flattened, leftovers))
        else:
            flattened = array

        # Now normalize, making the largest magnitude 1
        peak = np.max(np.abs(array))
        return flattened / peak

    def spectrum(self, t, nSamples, remove_dc=True):
        """
        Compute a spectrum from nSamples centered at time t
        """
        from spectrum import Spectrum
        tStart = t - nSamples // 2 * self.dt
        if tStart < self.t0:
            tStart = self.t0
        raw = self.values(tStart, nSamples)
        return Spectrum(raw, self.dt, remove_dc)



In [4]:
spec = Spectrogram("GEN3CH_4_009.dig")

In [5]:
# spec.plot(max_vel=5000, max_time = 50)
# plt.show

In [6]:
times, velocities, intensities = spec.slice((spec.time[10],spec.time[20]),(1000,4000))


In [7]:
velocities.shape

(636,)

In [8]:
len(intensities[:,2])

636

In [9]:
maxIndex = np.argmax(intensities[:,2])
print(maxIndex)
for i in range(0,2):
    print(velocities[np.argmax(intensities[:,i])])

204
1963.043212890625
1953.582763671875


In [21]:
def greedy(spectogram,startUS,endUS,startV,endV):
#     del times,velocities,intensities
    tstart = timeIndex(startUS,spectogram)
    tend = timeIndex(endUS,spectogram)
    
    vstart = velocityIndex(startV,spectogram)
    vend = velocityIndex(endV, spectogram)
    
    print('vstart,vend',vstart,vend)
    times,velocites,intensities = spec.slice((spec.time[tstart],spec.time[tend]),(spec.velocity[vstart],spec.velocity[vend]))
    print(velocities)
    greedyIndex = np.zeros(tend-tstart,dtype=int)
    
    #greedyVels is storing the index of the velocities
    greedyIndex[0] = int(np.argmax(intensities[:,0]))
#     print(greedyVels)
    i = 1
    while i != tend-tstart:
#         print(i)
#         print(int(greedyIndex[i-1]))
        greedyIndex[i] = greedyCompare(intensities[:,i],intensities[:,i+1],greedyIndex[i-1])
        i += 1
#     print( greedyIndex)
    Vcoords = np.zeros(tend-tstart)
    times = spectogram.time[tstart:tend+1]
#     print(coords)
    i = 0
    while i < tend-tstart:
#         print(i)
        Vcoords[i] = velocities[greedyIndex[i]]
#         print(i,greedyIndex[i],velocities[greedyIndex[i]])
        i += 1
    return times[:-1], Vcoords
        
        
        
        
        
        
def greedyCompare(time0intens, time1intens, start ):
    """ 
    Inputs: two arrays of intensities, at two times. indexes = velocity indexes
                start: the index in time0 of the current velocity selection
    Output: index of the greedy match
    verified 11/26
    """
#     print('start = ', start)
    oldVelocity = time0intens[start]
    nextOptions = []
    for i in range(0,3): #makes -1,0,1
        try:
            nextOptions += [abs(oldVelocity-time1intens[start+i])]
        except:
            nextOptions += [10000]
    return start+nextOptions.index(min(nextOptions))

def timeIndex(microS,spectrogram):
    seconds = microS*1e-6
    start = spectrogram.time[0]
    timeStep = abs(start-spectrogram.time[1])
    steps = (seconds-start)/timeStep
    steps = int(steps)
    return  steps-1


def velocityIndex(vel,spectrogram):
    start = spectrogram.velocity[0]
    vStep = abs(start-spectrogram.velocity[1])
#     print(vStep)
    vsteps = (vel-start)/vStep
    vsteps = int(vsteps)
    return  vsteps

        
        
t,v = greedy(spec,100,120,100,2000)

print(t.shape,v.shape)
     

vstart,vend 21 422
[ 998.07739258 1002.80761719 1007.5378418  1012.26806641 1016.99829102
 1021.72851562 1026.45874023 1031.18896484 1035.91918945 1040.64941406
 1045.37963867 1050.10986328 1054.84008789 1059.5703125  1064.30053711
 1069.03076172 1073.76098633 1078.49121094 1083.22143555 1087.95166016
 1092.68188477 1097.41210938 1102.14233398 1106.87255859 1111.6027832
 1116.33300781 1121.06323242 1125.79345703 1130.52368164 1135.25390625
 1139.98413086 1144.71435547 1149.44458008 1154.17480469 1158.9050293
 1163.63525391 1168.36547852 1173.09570312 1177.82592773 1182.55615234
 1187.28637695 1192.01660156 1196.74682617 1201.47705078 1206.20727539
 1210.9375     1215.66772461 1220.39794922 1225.12817383 1229.85839844
 1234.58862305 1239.31884766 1244.04907227 1248.77929688 1253.50952148
 1258.23974609 1262.9699707  1267.70019531 1272.43041992 1277.16064453
 1281.89086914 1286.62109375 1291.35131836 1296.08154297 1300.81176758
 1305.54199219 1310.2722168  1315.00244141 1319.73266602 132

In [19]:
gTimes,gVels = greedy(spec,11,30,2100,4000)
# fig = spec.plot(max_vel=5000, max_time = 50)
spec.plot(max_vel=5000, max_time = 50)

plt.plot(gTimes*1e6, gVels, color="red")
        
plt.show

vstart,vend 443 845
[ 998.07739258 1002.80761719 1007.5378418  1012.26806641 1016.99829102
 1021.72851562 1026.45874023 1031.18896484 1035.91918945 1040.64941406
 1045.37963867 1050.10986328 1054.84008789 1059.5703125  1064.30053711
 1069.03076172 1073.76098633 1078.49121094 1083.22143555 1087.95166016
 1092.68188477 1097.41210938 1102.14233398 1106.87255859 1111.6027832
 1116.33300781 1121.06323242 1125.79345703 1130.52368164 1135.25390625
 1139.98413086 1144.71435547 1149.44458008 1154.17480469 1158.9050293
 1163.63525391 1168.36547852 1173.09570312 1177.82592773 1182.55615234
 1187.28637695 1192.01660156 1196.74682617 1201.47705078 1206.20727539
 1210.9375     1215.66772461 1220.39794922 1225.12817383 1229.85839844
 1234.58862305 1239.31884766 1244.04907227 1248.77929688 1253.50952148
 1258.23974609 1262.9699707  1267.70019531 1272.43041992 1277.16064453
 1281.89086914 1286.62109375 1291.35131836 1296.08154297 1300.81176758
 1305.54199219 1310.2722168  1315.00244141 1319.73266602 13

IndexError: index 1 is out of bounds for axis 1 with size 1

In [12]:
def velocityIndex(vel,spectrogram):
    start = spectrogram.velocity[0]
    vStep = abs(start-spectrogram.velocity[1])
#     print(vStep)
    vsteps = (vel-start)/vStep
    vsteps = int(vsteps)
    return  vsteps


velocityIndex(200,spec)

42

In [13]:
gTimes,gVels = greedy(spec,10,20,1000,2000)
gTimes+spec.time[10]

vstart,vend 211 422
[ 998.07739258 1002.80761719 1007.5378418  1012.26806641 1016.99829102
 1021.72851562 1026.45874023 1031.18896484 1035.91918945 1040.64941406
 1045.37963867 1050.10986328 1054.84008789 1059.5703125  1064.30053711
 1069.03076172 1073.76098633 1078.49121094 1083.22143555 1087.95166016
 1092.68188477 1097.41210938 1102.14233398 1106.87255859 1111.6027832
 1116.33300781 1121.06323242 1125.79345703 1130.52368164 1135.25390625
 1139.98413086 1144.71435547 1149.44458008 1154.17480469 1158.9050293
 1163.63525391 1168.36547852 1173.09570312 1177.82592773 1182.55615234
 1187.28637695 1192.01660156 1196.74682617 1201.47705078 1206.20727539
 1210.9375     1215.66772461 1220.39794922 1225.12817383 1229.85839844
 1234.58862305 1239.31884766 1244.04907227 1248.77929688 1253.50952148
 1258.23974609 1262.9699707  1267.70019531 1272.43041992 1277.16064453
 1281.89086914 1286.62109375 1291.35131836 1296.08154297 1300.81176758
 1305.54199219 1310.2722168  1315.00244141 1319.73266602 13

array([-0.00018885, -0.00018873, -0.00018861, -0.00018848, -0.00018836,
       -0.00018824, -0.00018811, -0.00018799, -0.00018787, -0.00018775,
       -0.00018762, -0.0001875 , -0.00018738, -0.00018725, -0.00018713,
       -0.00018701, -0.00018689, -0.00018676, -0.00018664, -0.00018652,
       -0.00018639, -0.00018627, -0.00018615, -0.00018602, -0.0001859 ,
       -0.00018578, -0.00018566, -0.00018553, -0.00018541, -0.00018529,
       -0.00018516, -0.00018504, -0.00018492, -0.0001848 , -0.00018467,
       -0.00018455, -0.00018443, -0.0001843 , -0.00018418, -0.00018406,
       -0.00018394, -0.00018381, -0.00018369, -0.00018357, -0.00018344,
       -0.00018332, -0.0001832 , -0.00018308, -0.00018295, -0.00018283,
       -0.00018271, -0.00018258, -0.00018246, -0.00018234, -0.00018222,
       -0.00018209, -0.00018197, -0.00018185, -0.00018172, -0.0001816 ,
       -0.00018148, -0.00018136, -0.00018123, -0.00018111, -0.00018099,
       -0.00018086, -0.00018074, -0.00018062, -0.0001805 , -0.00

In [14]:

def greedyCompare(time0intens, time1intens, start ):
    """ 
    Inputs: two arrays of intensities, at two times. indexes = velocity indexes
                start: the index in time0 of the current velocity selection
    Output: index of the greedy match
    """
    oldVelocity = time0intens[start]
    nextOptions = []
    for i in range(0,3): #makes -1,0,1
        nextOptions += [abs(oldVelocity-time1intens[start+i])]
    return start+nextOptions.index(min(nextOptions))


# intensities[:,0]
greedyCompare(intensities[:,0],intensities[:,1],7)
print(intensities[7,0])
print(intensities[9,1])
np.argmax(intensities[:,0])
intensities[150,0]

-23.48290445685165
-9.855821879045914


-19.417692502852894

In [15]:
times,velocites,intensities = spec.slice((spec.time[0],spec.time[10]),(1000,2000))
intensities[np.argmax(intensities[:,0]),0]

22.689318753215222